<img src="img/dsci513_header2.png" width="600">

# Lecture 8: Advanced queries in MongoDB

## Lecture outline

- Operators in MongoDB
- Query conditionals
- Querying arrays and sub-documents
- Aggregation pipelines

In [1]:
from pymongo import MongoClient
import json

with open('data/credentials_mongodb.json') as f:
    login = json.load(f)

client = MongoClient(**login)

<br><br><br>

## `$` and operators in MongoDB

In MongoDB, operators are denoted with a dollar sign `$`. For example, the `$sum` operator which is used in aggregation pipelines, or comparison operators such as `$gte` which is equivalent to `>=` in SQL or Python.

<br><br><br>

**Note that:**

The dollar sign `$` also has another use case: if a field needs to appear as a value, we need to reference that using a `$`. For example, I'll show you later in this lecture that if you want to rename a field in the output, you can have something like this in the projection stage of a pipeline:

```
{'$project': {'duration': '$runtime'}}
```

Had I used `{'duration': 'runtime'}`, the `runtime` would have been interpreted as a literal string, not the actual values in the `runtime` field. You'll also see this used in the `_id` field of a `$group` stage for a similar reason.

## Comparison operators

### `$gt`, `$gte`, `$lt`, `$lte`

These operators have the same meaning as `>`, `>=`, `<`, and `<=` in SQL or Python. For example, while a filter `{'runtime': 200}` would return documents whose runtime is exactly 200 minutes, `{'runtime': {'$gte': 200}}` would return documents whose runtime is greater that 200 minutes.

<br><br><br>

**Example:** Return the title, runtime, and production year of 5 movies with a runtime of 200 minutes or greater.

In [2]:
list(
    client['sample_mflix']['movies'].find(
        filter={'runtime': {'$gte': 200}},
        projection={'_id': 0, 'title': 1, 'runtime': 1, 'year': 1},
        limit=5
    )
)

[{'runtime': 399, 'title': 'Les vampires', 'year': 1915},
 {'runtime': 240, 'title': 'Napoleon', 'year': 1927},
 {'runtime': 281, 'title': 'Les Misèrables', 'year': 1934},
 {'runtime': 245, 'title': 'Flash Gordon', 'year': 1936},
 {'runtime': 238, 'title': 'Gone with the Wind', 'year': 1939}]

<br><br><br>

**Example:** How many movies are there with a runtime of 200 minutes or greater?

In [3]:
client['sample_mflix']['movies'].count_documents(filter={'runtime': {'$gte': 200}})

227

<br><br><br>

### `$ne`

The `$ne` (not equal) operator has the same meaning as `<>` in SQL or `!=` in Python.

<br><br><br>

**Example:** Find the title and the type of 5 documents in the `movies` collection that are not of type `movie`.

In [4]:
list(
    client['sample_mflix']['movies'].find(
        filter={'type': {'$ne': 'movie'}},
        projection={'_id': 0, 'title': 1, 'type': 1},
        limit=5
    )
)

[{'title': 'The Forsyte Saga', 'type': 'series'},
 {'title': 'Scenes from a Marriage', 'type': 'series'},
 {'title': 'Ironiya sudby, ili S legkim parom!', 'type': 'series'},
 {'title': 'I, Claudius', 'type': 'series'},
 {'title': 'Sybil', 'type': 'series'}]

Note that the `type` field of none of the returned documents is `movie` (all of them are `series` because that's the only other type that exists in the documents of the `movies` collection).

<br><br><br>

### `$in`, `$nin`

These two operators are equivalent to `IN` and `NOT IN` in SQL, or `in` and `not in` in Python. They are used to check if the value of a field is equal (or not equal) to any value in a given list. What these operators do can also be imitated with `$or` or `$nor`, but these are more concise.

**Example:** Return the title, production year, and the cast of these movies: The Sixth Sense, Imitation Game, The Red Violin, Match Point, Forrest Gump.

In [5]:
list(
    client["sample_mflix"]["movies"].find(
        filter={
            "title": {
                "$in": [
                    "The Sixth Sense",
                    "Imitation Game",
                    "The Red Violin",
                    "Match Point",
                    "Forrest Gump",
                ],
            },
        },
        projection={"_id": 0, "title": 1, "cast": 1, "year": 1},
        limit=5,
    )
)

[{'year': 1994,
  'title': 'Forrest Gump',
  'cast': ['Tom Hanks',
   'Rebecca Williams',
   'Sally Field',
   'Michael Conner Humphreys']},
 {'cast': ['Carlo Cecchi',
   'Irene Grazioli',
   'Anita Laurenzi',
   'Tommaso Puntelli'],
  'title': 'The Red Violin',
  'year': 1998},
 {'year': 1999,
  'title': 'The Sixth Sense',
  'cast': ['Bruce Willis',
   'Haley Joel Osment',
   'Toni Collette',
   'Olivia Williams']},
 {'year': 2005,
  'title': 'Match Point',
  'cast': ['Jonathan Rhys Meyers',
   'Alexander Armstrong',
   'Paul Kaye',
   'Matthew Goode']}]

<br><br><br>

**Example:** Find the number of movies that are not available in any of these languages: English, French, Italian or German.

In [6]:
client["sample_mflix"]["movies"].count_documents(
    filter={'languages': {'$nin': ['English', 'French', 'German', 'Italian']}}
)

4888

<br><br><br>

## Logical operators

### Implicit AND, `$and`

In MongoDB, specifying multiple conditions on a field or multiple fields implies an implicit logical AND and we don't need to explicitly need to use `$and`. For example, in these filters

```
{'year': {'$gte': 2000, '$lte': 2010}}
```
or
```
{'year': 2010, 'directors': 'Woody Allen'}
```
a logical AND is implied between the multiple given conditions.

<br><br><br>

There are situations where we need the actual boolean result of the logical AND operation. For example, this projection stage will create a field in the output documents called `high_rated_voted` which evaluates to `true` if both the IMDB rating and the number of votes are high enough, and to `false` otherwise:

```
$project: {
            'title': 1,
            'year': 1,
            'high_rated_voted': {
                '$and': [ 
                    { $gt: [ $imdb.rating', 8 ] },
                    { '$gt': [ '$imdb.votes', 100000 ] } 
                    ]
             }
          }
```

<br><br><br>

**Example:** Using the `sample_supplies` database, return 2 documents associated with sales that are made in Seattle and used a coupon. Exclude the `items` field from the results.

In [7]:
list(
    client["sample_supplies"]["sales"].find(
        filter={
            'storeLocation': 'Seattle',
            'couponUsed': True
        },
        projection={"_id": 0, "items": 0},
        limit=2,
    )
)

[{'saleDate': datetime.datetime(2016, 5, 16, 3, 43, 15, 808000),
  'storeLocation': 'Seattle',
  'customer': {'gender': 'F',
   'age': 26,
   'email': 'kupeen@gareha.ne',
   'satisfaction': 5},
  'couponUsed': True,
  'purchaseMethod': 'In store'},
 {'saleDate': datetime.datetime(2016, 10, 22, 18, 50, 44, 216000),
  'storeLocation': 'Seattle',
  'customer': {'gender': 'F',
   'age': 52,
   'email': 'nupzig@apubu.hu',
   'satisfaction': 3},
  'couponUsed': True,
  'purchaseMethod': 'In store'}]

The above query is equivalent to:

In [8]:
list(
    client["sample_supplies"]["sales"].find(
        filter={
            '$and': [   
                {'storeLocation': 'Seattle'},
                {'couponUsed': True}
            ]
        },
        projection={"_id": 0, "items": 0},
        limit=2,
    )
)

[{'saleDate': datetime.datetime(2016, 5, 16, 3, 43, 15, 808000),
  'storeLocation': 'Seattle',
  'customer': {'gender': 'F',
   'age': 26,
   'email': 'kupeen@gareha.ne',
   'satisfaction': 5},
  'couponUsed': True,
  'purchaseMethod': 'In store'},
 {'saleDate': datetime.datetime(2016, 10, 22, 18, 50, 44, 216000),
  'storeLocation': 'Seattle',
  'customer': {'gender': 'F',
   'age': 52,
   'email': 'nupzig@apubu.hu',
   'satisfaction': 3},
  'couponUsed': True,
  'purchaseMethod': 'In store'}]

<br><br><br>

**Example:** Using the `sample_supplies` database, find the number of sales made between October 1, 2014 and December 1, 2014.

In [9]:
import datetime

client["sample_supplies"]["sales"].count_documents(
        filter={
            'saleDate': {
                '$gte': datetime.datetime(2014, 10, 1),
                '$lte': datetime.datetime(2014, 12, 1)
            }
        }
)

153

<br><br><br>

> **Note:** When specifying multiple conditions for a field, all conditions should be put into a single query document for that field (or an `$and` operator should be used). For example, the following query only checks the **last condition on the field** and returns unexpected results:

In [10]:
import datetime

client["sample_supplies"]["sales"].count_documents(
        filter={
            'saleDate': {'$gte': datetime.datetime(2014, 10, 1)},
            'saleDate': {'$lte': datetime.datetime(2014, 12, 1)}
    }
)

1861

<br><br><br>

### `$or`, `$nor`

These are general logical OR/NOR operators. The difference with `$in`/`$nin` is that `$in`/`$nin` can be used for an equality condition on a single field, but `$or`/`$nor` are general and can be used with any boolean expression.

<br><br><br>

**Example:** Using the `sample_supplies` database, find 5 sales that were made in New York and were either paid by phone or didn't use a coupon.

In [11]:
list(
    client["sample_supplies"]["sales"].find(
        filter={
            'storeLocation': 'New York',
            '$or': [
                {'purchaseMethod': 'Phone'},
                {'couponUsed': True}
            ]
        },
        projection={"_id": 0, "items": 0},
        limit=5,
    )
)

[{'saleDate': datetime.datetime(2017, 3, 21, 1, 54, 26, 657000),
  'storeLocation': 'New York',
  'customer': {'gender': 'M',
   'age': 26,
   'email': 'rapifoozi@viupoen.bb',
   'satisfaction': 5},
  'couponUsed': True,
  'purchaseMethod': 'In store'},
 {'saleDate': datetime.datetime(2013, 8, 21, 9, 36, 7, 188000),
  'storeLocation': 'New York',
  'customer': {'gender': 'F',
   'age': 53,
   'email': 'se@nacwev.an',
   'satisfaction': 4},
  'couponUsed': False,
  'purchaseMethod': 'Phone'},
 {'saleDate': datetime.datetime(2017, 9, 2, 15, 42, 22, 167000),
  'storeLocation': 'New York',
  'customer': {'gender': 'M',
   'age': 34,
   'email': 'bubbecgu@odidecned.tf',
   'satisfaction': 3},
  'couponUsed': False,
  'purchaseMethod': 'Phone'},
 {'saleDate': datetime.datetime(2015, 3, 6, 19, 13, 35, 155000),
  'storeLocation': 'New York',
  'customer': {'gender': 'F',
   'age': 42,
   'email': 'jecosab@copfatma.af',
   'satisfaction': 3},
  'couponUsed': False,
  'purchaseMethod': 'Phone'},

<br><br><br>

### `$not`

This is similar to `NOT` in SQL or `not` in Python, and is used to negate a boolean expression. This could be useful, for example, when you want to match field values that **do not** follow a particular regex pattern.

<br><br><br>

## Field existence with `$exists`

One of the hallmarks of document-based NoSQL databases is that **their schema is flexible**, meaning that not all documents need to have the same fields. This is why sometimes we need to check for the existence of a field before trying to use it. This is done using the `$exists` operator in MongoDB.

In the following example, I want to select documents for movies that are not in English. However, I should make sure that `languages` field actually exists, because otherwise I might filter out English movies that simply don't have the `languages` field:

In [12]:
list(
    client["sample_mflix"]["movies"].find(
        filter={
            'languages': {
                '$ne': 'English',
                '$exists': True  # comment this line to see the difference
            }
        },
        projection={'_id': 0, 'languages': 1, 'title': 1},
        limit=10,
    )
)

[{'title': 'Les vampires', 'languages': ['French']},
 {'title': 'Nosferatu', 'languages': ['German']},
 {'title': 'Battleship Potemkin', 'languages': ['Russian']},
 {'title': 'Metropolis', 'languages': ['German']},
 {'title': "Pandora's Box", 'languages': ['German']},
 {'title': 'The Passion of Joan of Arc', 'languages': ['French']},
 {'title': 'Storm Over Asia', 'languages': ['Russian']},
 {'title': 'Asphalt', 'languages': ['German']},
 {'title': 'David Golder', 'languages': ['French']},
 {'title': 'The Blood of a Poet', 'languages': ['French']}]

<br><br><br>

## Pattern matching with `$regex`

Pattern matching using regex is done in pretty much the same way as in Postgres. Because MongoDB uses JavaScript notation for regex by default, in Compass or `mongosh` patterns need to be enclosed in fore-slashes, e.g. `/pattern/`. In `pymongo` however, you can use regex patterns without the fore-slashes, just like in SQL.

<br><br><br>

**Example:** Return 5 documents for movies whose title contains 3 digits (and 3 digits only) at the end (e.g. UBC 333).

In [13]:
list(
    client["sample_mflix"]["movies"].find(
        filter={
            'title': {'$regex': '[^\d]\d{3}$'}
        },
        projection={'_id': 0, 'title': 1},
        limit=5,
    )
)

[{'title': 'Call Northside 777'},
 {'title': 'Mister 880'},
 {'title': 'Shree 420'},
 {'title': 'Stop Train 349'},
 {'title': 'Fahrenheit 451'}]

<br><br><br>

**Example:** Return 10 documents for movies whose title does not contain any letters (case insensitive).

In [14]:
list(
    client["sample_mflix"]["movies"].find(
        filter={
            'title': {'$not': {'$regex': '(?i)[a-z]'}}
        },
        projection={'_id': 0, 'title': 1},
        limit=10,
    )
)

[{'title': '8è'},
 {'title': '$'},
 {'title': '1776'},
 {'title': '1900'},
 {'title': '10'},
 {'title': '2010'},
 {'title': '1969'},
 {'title': '12:01'},
 {'title': '1-900'},
 {'title': '54'}]

<br><br><br>

## Querying sub-documents

In [15]:
client['mds']['instructors'].drop()

Let's first create a toy database called `mds`:

In [16]:
if client['mds']['instructors'].count_documents({}) == 0:
    
    client['mds']['instructors'].insert_many([
        {
            'name': 'Arman',
            'department': {
                'name': 'Computer Science',
                'campus': 'Vancouver'
            },
            'courses': [
                {
                    'name': 'Algorithms & Data Structures',
                    'code': 512
                },
                {
                    'name': 'Descriptive Statistics and Probability',
                    'code': 551
                },
            ]
        },
        {
            'name': 'Rodolfo',
            'department': {
                'name': 'Statistics',
                'campus': 'Vancouver'
            },
            'courses': [
                {
                    'name': 'Programming for Data Manipulation',
                    'code': 523
                },
                {
                    'name': 'Data Science Workflows',
                    'code': 522
                },
                {
                    'name': 'Collaborative Software Development',
                    'code': 524
                },
            ]
        },
        {
            'name': 'Alexi',
            'department': {
                'name': 'Statistics',
                'campus': 'Vancouver'
            },
            'courses': [
                {
                    'name': 'Statistical Inference and Computation',
                    'code': 552
                },
                {
                    'name': 'Regression II',
                    'code': 562
                },
            ]
        }
    ])

Sub-documents can be queried for either for an **exact match** or for **particular sub-fields**.

The following query wouldn't work because it looks for a sub-document `'department': {'name': 'Statistics'}` (exact match, no more no less) which does not exist:

In [17]:
list(
    client['mds']['instructors'].find(
        filter={
            'department': {'name': 'Statistics'}
        }
    )
)

[]

But if we supply the full `department` sub-document:

In [18]:
list(
    client['mds']['instructors'].find(
        filter={
            'department': {'name': 'Statistics', 'campus': 'Vancouver'}
        }
    )
)

[{'_id': ObjectId('6391cf14210a99d148d3180e'),
  'name': 'Rodolfo',
  'department': {'name': 'Statistics', 'campus': 'Vancouver'},
  'courses': [{'name': 'Programming for Data Manipulation', 'code': 523},
   {'name': 'Data Science Workflows', 'code': 522},
   {'name': 'Collaborative Software Development', 'code': 524}]},
 {'_id': ObjectId('6391cf14210a99d148d3180f'),
  'name': 'Alexi',
  'department': {'name': 'Statistics', 'campus': 'Vancouver'},
  'courses': [{'name': 'Statistical Inference and Computation', 'code': 552},
   {'name': 'Regression II', 'code': 562}]}]

<br><br><br>

However, we can look for particular values for sub-fields directly using **the dot notation**.

For example, if we want to find instructors from the Statistics department, we can use the following query:

In [19]:
list(
    client['mds']['instructors'].find(
        filter={
            'department.name': 'Statistics'
        }
    )
)

[{'_id': ObjectId('6391cf14210a99d148d3180e'),
  'name': 'Rodolfo',
  'department': {'name': 'Statistics', 'campus': 'Vancouver'},
  'courses': [{'name': 'Programming for Data Manipulation', 'code': 523},
   {'name': 'Data Science Workflows', 'code': 522},
   {'name': 'Collaborative Software Development', 'code': 524}]},
 {'_id': ObjectId('6391cf14210a99d148d3180f'),
  'name': 'Alexi',
  'department': {'name': 'Statistics', 'campus': 'Vancouver'},
  'courses': [{'name': 'Statistical Inference and Computation', 'code': 552},
   {'name': 'Regression II', 'code': 562}]}]

In this way, if over time other fields are added to the `department` field, our query will still be robust and run without problems.

<br><br><br>

## Querying arrays

Similar to sub-documents, arrays can either be matched exactly or partially.

If we're looking for a **single value** inside an array, we can write our query **as if the array field was a regular simple field**.

For example, the following query returns movies that are available in French, among other languages:

In [20]:
list(
    client["sample_mflix"]["movies"].find(
        filter={'languages': 'French'},
        projection={'_id': 0, 'languages': 1, 'title': 1},
        limit=10,
    )
)

[{'title': 'Les vampires', 'languages': ['French']},
 {'title': 'The Passion of Joan of Arc', 'languages': ['French']},
 {'title': 'All Quiet on the Western Front',
  'languages': ['English', 'French', 'German', 'Latin']},
 {'title': 'David Golder', 'languages': ['French']},
 {'title': 'The Divorcee', 'languages': ['English', 'French']},
 {'title': 'Morocco',
  'languages': ['English', 'French', 'Spanish', 'Arabic', 'Italian']},
 {'title': 'The Blood of a Poet', 'languages': ['French']},
 {'title': 'Under the Roofs of Paris', 'languages': ['French', 'Romanian']},
 {'title': 'Cimarron', 'languages': ['English', 'French']},
 {'title': 'Comradeship', 'languages': ['French', 'German']}]

Note that the `languages` field in all of the above documents contains French, while other languages are also occasionally found in the array.

<br><br><br>

Things are different if we're looking for **more than one element**. If the desired elements are specified inside `[]`, then MongoDB looks for an exact match.

For example, with the following query:

In [21]:
list(
    client["sample_mflix"]["movies"].find(
        filter={
            'languages': ['English', 'French']  # order matters here
            # 'languages': ['French', 'English']
        },
        projection={'_id': 0, 'languages': 1, 'title': 1},
        limit=10,
    )
)

[{'title': 'The Divorcee', 'languages': ['English', 'French']},
 {'title': 'Cimarron', 'languages': ['English', 'French']},
 {'title': 'The Sin of Madelon Claudet', 'languages': ['English', 'French']},
 {'title': 'Forbidden', 'languages': ['English', 'French']},
 {'title': 'Baby Face', 'languages': ['English', 'French']},
 {'title': 'Footlight Parade', 'languages': ['English', 'French']},
 {'title': 'Going Hollywood', 'languages': ['English', 'French']},
 {'title': 'Death Takes a Holiday', 'languages': ['English', 'French']},
 {'title': 'Becky Sharp', 'languages': ['English', 'French']},
 {'title': 'Folies Bergère de Paris', 'languages': ['English', 'French']}]

only those documents will be returned that **only and only** have `['English', 'French']` in their `'languages'`, **in the exact same order**. You can switch the order of the languages to see that you get different results.

<br><br><br>

We can also query an array by the **index** of its elements, just like in Python.

Here, I'm looking for movies which have Italian listed as the first language in their `languages` field:

In [22]:
list(
    client["sample_mflix"]["movies"].find(
        filter={
            'languages.0': 'Italian'
        },
        projection={'_id': 0, 'languages': 1, 'title': 1},
        limit=10,
    )
)

[{'title': "Everybody's Woman", 'languages': ['Italian']},
 {'title': 'The Siege of the Alcazar', 'languages': ['Italian']},
 {'title': 'The White Ship', 'languages': ['Italian']},
 {'title': 'Ossessione', 'languages': ['Italian']},
 {'title': 'The Testimony', 'languages': ['Italian']},
 {'title': 'The Bandit', 'languages': ['Italian', 'German', 'English']},
 {'title': 'La porta del cielo', 'languages': ['Italian']},
 {'title': 'Paisan',
  'languages': ['Italian', 'English', 'German', 'Sicilian']},
 {'title': 'Rome, Open City', 'languages': ['Italian', 'German', 'Latin']},
 {'title': 'Shoeshine', 'languages': ['Italian', 'English']}]

<br><br><br>

### `$size`

The `$size` operator checks for the number of elements inside an array.

For example, suppose that we want to return all movies that are have 2 directors:

In [23]:
list(
    client["sample_mflix"]["movies"].find(
        filter={
            'directors': {'$size': 2}
        },
        projection={'_id': 0, 'directors': 1, 'title': 1},
        limit=10,
    )
)

[{'title': 'Winsor McCay, the Famous Cartoonist of the N.Y. Herald and His Moving Comics',
  'directors': ['Winsor McCay', 'J. Stuart Blackton']},
 {'title': 'The Perils of Pauline',
  'directors': ['Louis J. Gasnier', 'Donald MacKenzie']},
 {'title': 'Where Are My Children?',
  'directors': ['Phillips Smalley', 'Lois Weber']},
 {'title': 'From Hand to Mouth',
  'directors': ['Alfred J. Goulding', 'Hal Roach']},
 {'title': 'The Last of the Mohicans',
  'directors': ['Clarence Brown', 'Maurice Tourneur']},
 {'title': 'One Week', 'directors': ['Edward F. Cline', 'Buster Keaton']},
 {'title': 'The Saphead', 'directors': ['Herbert Blachè', 'Winchell Smith']},
 {'title': 'Now or Never', 'directors': ['Fred C. Newmeyer', 'Hal Roach']},
 {'title': 'Cops', 'directors': ['Edward F. Cline', 'Buster Keaton']},
 {'title': 'Salomè', 'directors': ['Charles Bryant', 'Alla Nazimova']}]

<br><br><br>

### `$all`

Now suppose that we want to find movies that are available in both English and French, among other languages, and we don't care about the order in which they appear. For this situation, we can use the `$all` operator to match arrays that contain all listed values among other values, in no particular order:

In [24]:
list(
    client["sample_mflix"]["movies"].find(
        filter={
            'languages': {'$all': ['English', 'French']}
        },
        projection={'_id': 0, 'languages': 1, 'title': 1},
        limit=10,
    )
)

[{'title': 'All Quiet on the Western Front',
  'languages': ['English', 'French', 'German', 'Latin']},
 {'title': 'The Divorcee', 'languages': ['English', 'French']},
 {'title': 'Morocco',
  'languages': ['English', 'French', 'Spanish', 'Arabic', 'Italian']},
 {'title': 'Cimarron', 'languages': ['English', 'French']},
 {'title': 'The Sin of Madelon Claudet', 'languages': ['English', 'French']},
 {'title': 'Forbidden', 'languages': ['English', 'French']},
 {'title': 'Freaks', 'languages': ['English', 'German', 'French']},
 {'title': 'The Mummy', 'languages': ['English', 'Arabic', 'French']},
 {'title': 'Shanghai Express',
  'languages': ['English', 'French', 'Cantonese', 'German']},
 {'title': 'Baby Face', 'languages': ['English', 'French']}]

Note that the `languages` field of all of the above documents contains English and French **in no particular order**, in addition to other languages that might happen to exist in the same array.

<br><br><br>

### `$elemMatch`

We can best see the use case of this operator with an example. Consider these documents in the `students` collection of the `mds` database:

In [25]:
client['mds']['students'].drop()

In [26]:
if client['mds']['students'].count_documents({}) == 0:
    
    client['mds']['students'].insert_many([
        {
            'name': 'Quan',
            'grades': [79, 87, 97]
        },
        {
            'name': 'Varada',
            'grades': [75, 82, 90]
        },
        {
            'name': 'Florencia',
            'grades': [92, 93, 77]
        }
    ])

Suppose that we want to retrieve the document for students who have a grade in the `[80, 85]` range. In our collection, only the student named "Varada" has a grade in that range (i.e. 82). Let's see if the following query works:

In [27]:
list(
    client['mds']['students'].find(
        filter={'grades': {'$gte': 80, '$lte': 85}}
    )
)

[{'_id': ObjectId('6391cf15210a99d148d31810'),
  'name': 'Quan',
  'grades': [79, 87, 97]},
 {'_id': ObjectId('6391cf15210a99d148d31811'),
  'name': 'Varada',
  'grades': [75, 82, 90]},
 {'_id': ObjectId('6391cf15210a99d148d31812'),
  'name': 'Florencia',
  'grades': [92, 93, 77]}]

Oops, it returned all documents!

The reason is that when multiple conditions are defined on an array, MongoDB checks the array **as a whole**. If there is at least one match for each condition in the whole array, then that document will be returned.

In order to force MongoDB to check the conditions on **each** individual element, we need to use `$elemMatch`:

In [28]:
list(
    client['mds']['students'].find(
        filter={'grades': {'$elemMatch': {'$gte': 80, '$lte': 85}}}
    )
)

[{'_id': ObjectId('6391cf15210a99d148d31811'),
  'name': 'Varada',
  'grades': [75, 82, 90]}]

<br><br><br>

### Arrays of sub-documents

Querying sub-documents nested inside an array works a lot like querying arrays with simple elements. Let's take a look at the `instructors` collection of the `mds` database we created earlier:

In [29]:
list(
    client['mds']['instructors'].find({})
)

[{'_id': ObjectId('6391cf14210a99d148d3180d'),
  'name': 'Arman',
  'department': {'name': 'Computer Science', 'campus': 'Vancouver'},
  'courses': [{'name': 'Algorithms & Data Structures', 'code': 512},
   {'name': 'Descriptive Statistics and Probability', 'code': 551}]},
 {'_id': ObjectId('6391cf14210a99d148d3180e'),
  'name': 'Rodolfo',
  'department': {'name': 'Statistics', 'campus': 'Vancouver'},
  'courses': [{'name': 'Programming for Data Manipulation', 'code': 523},
   {'name': 'Data Science Workflows', 'code': 522},
   {'name': 'Collaborative Software Development', 'code': 524}]},
 {'_id': ObjectId('6391cf14210a99d148d3180f'),
  'name': 'Alexi',
  'department': {'name': 'Statistics', 'campus': 'Vancouver'},
  'courses': [{'name': 'Statistical Inference and Computation', 'code': 552},
   {'name': 'Regression II', 'code': 562}]}]

<br><br><br>

Let's find instructor(s) who teach DSCI 512:

In [30]:
list(
    client['mds']['instructors'].find(
        filter={'courses': {'name': 'Algorithms & Data Structures', 'code': 512}}
    )
)

[{'_id': ObjectId('6391cf14210a99d148d3180d'),
  'name': 'Arman',
  'department': {'name': 'Computer Science', 'campus': 'Vancouver'},
  'courses': [{'name': 'Algorithms & Data Structures', 'code': 512},
   {'name': 'Descriptive Statistics and Probability', 'code': 551}]}]

Note that the following query fails:

In [31]:
list(
    client['mds']['instructors'].find(
        filter={'courses': {'code': 512}}
    )
)

[]

because just like arrays, a condition like `{'code': 512}` has to match **an entire sub-document**.

<br><br><br>

The easier way is to use the dot notation for reaching into the sub-documents in the array and place a condition on a particular sub-field

In [32]:
list(
    client['mds']['instructors'].find(
        filter={'courses.code': 512}
    )
)

[{'_id': ObjectId('6391cf14210a99d148d3180d'),
  'name': 'Arman',
  'department': {'name': 'Computer Science', 'campus': 'Vancouver'},
  'courses': [{'name': 'Algorithms & Data Structures', 'code': 512},
   {'name': 'Descriptive Statistics and Probability', 'code': 551}]}]

<br><br><br>

Just like for arrays, **mixed** conditions are checked for **all elements of the array**.

For example, in the following although the course code for "Algorithms & Data Structures" **is not** 551, the document for Arman is still returned because each condition matches at least one element in the array:

In [33]:
list(
    client['mds']['instructors'].find(
        filter={'courses.code': 551, 'courses.name': {'$regex': 'Algorithms'}}
    )
)

[{'_id': ObjectId('6391cf14210a99d148d3180d'),
  'name': 'Arman',
  'department': {'name': 'Computer Science', 'campus': 'Vancouver'},
  'courses': [{'name': 'Algorithms & Data Structures', 'code': 512},
   {'name': 'Descriptive Statistics and Probability', 'code': 551}]}]

We can use `$elemMatch` here too to do an element-by-element condition check:

In [34]:
list(
    client['mds']['instructors'].find(
        filter={'courses': {'$elemMatch': {'code': 551, 'name': {'$regex': 'Algorithms'}}}}
    )
)

[]

And this time it correctly returns nothing, because no document exists that have "Algorithms & Data Structures" listed with the code 551.

<br><br><br>

## Aggregation pipelines

The aggregation framework is one of the most significant features of MongoDB. The aggregation frame work provides an intuitive way for **step-by-step processing of data in the form of multiple stages of a pipeline**.

<img src="img/lecture8/agg_pipe.png" width="700">

([image source](https://stackoverflow.com/a/39518920))

As a matter of fact, all of the queries we have written so far can be written in the context of an aggregation pipeline, with each operation (e.g. filtering, projection, limiting, skipping, counting, etc.) becoming one stage of the pipeline.

<br><br><br>

The syntax of an aggregation pipeline in `pymongo` looks like the following:

```python
client['database']['collection'].aggregate(
    [
        stage1,
        stage2,
        .
        .
        .
    ]
)
```

We can also create an aggregation pipeline using the _Aggregation_ tab in MongoDB Compass. The good thing about using Compass for constructing pipelines is that you can inspect the intermediate results of each stage.

<br><br><br>

There are several different stages of a pipeline offered by MongoDB (see a list [here](https://docs.mongodb.com/manual/reference/operator/aggregation-pipeline/)). Here, we cover the most essential ones.

Also, you can find a comprehensive side-by-side comparison of SQL aggregations and MongoDB aggregations [here](https://docs.mongodb.com/manual/reference/sql-aggregation-comparison/) in the MongoDB's documentations.

<br><br><br>

### `$match`

The `$match` stage of a pipeline is in charge of filtering the data. The conditions specified in this stage are similar to those we used for the `filter=` argument before.

In [35]:
list(
    client['mds']['instructors'].aggregate(
        [
            {"$match": {'department.name': 'Statistics'}}
        ]
    )
)

[{'_id': ObjectId('6391cf14210a99d148d3180e'),
  'name': 'Rodolfo',
  'department': {'name': 'Statistics', 'campus': 'Vancouver'},
  'courses': [{'name': 'Programming for Data Manipulation', 'code': 523},
   {'name': 'Data Science Workflows', 'code': 522},
   {'name': 'Collaborative Software Development', 'code': 524}]},
 {'_id': ObjectId('6391cf14210a99d148d3180f'),
  'name': 'Alexi',
  'department': {'name': 'Statistics', 'campus': 'Vancouver'},
  'courses': [{'name': 'Statistical Inference and Computation', 'code': 552},
   {'name': 'Regression II', 'code': 562}]}]

<br><br><br>

### `$project`

We have seen before that projection is used to include or exclude particular attributes in the resulting documents:

In [36]:
list(
    client['mds']['instructors'].aggregate(
        [
            {'$match': {'department.name': 'Statistics'}},
            {'$project': {'_id': 0, 'name': 1}}
        ]
    )
)

[{'name': 'Rodolfo'}, {'name': 'Alexi'}]

<br><br><br>

One potentially useful thing we can do using the `$project` stage is **sub-field promotion**.

For example, suppose that we want the name of each instructor, as well as the name of their department to appear as a top-level field with a custom name. We can achieve this using the following:

In [37]:
list(
    client['mds']['instructors'].aggregate(
        [
            {'$project': {'_id': 0, 'name': 1, 'dept_name': '$department.name'}}
        ]
    )
)

[{'name': 'Arman', 'dept_name': 'Computer Science'},
 {'name': 'Rodolfo', 'dept_name': 'Statistics'},
 {'name': 'Alexi', 'dept_name': 'Statistics'}]

> Note the usage of `$` in `'$department.name'`. Here we're referring to the values of the `department.name` field. If we don't prefix `department.name` with `$`, we'll get a static string in the output.

<br><br><br>

### `$sort`, `$limit`, `$skip`

We've already seen examples of sorting, skipping and limiting results. Here's an example within the context of aggregation pipelines:

In [38]:
list(
    client["sample_mflix"]["movies"].aggregate(
        [
            {"$match": {"imdb.votes": {"$ne": ""}}},
            {"$project": {"_id": 0, "title": 1, "imdb.votes": 1}},
            {"$sort": {"imdb.votes": -1}},
            {'$limit': 10},
        ]
    )
)

[{'imdb': {'votes': 1521105}, 'title': 'The Shawshank Redemption'},
 {'imdb': {'votes': 1513145}, 'title': 'The Shawshank Redemption'},
 {'imdb': {'votes': 1495351}, 'title': 'The Dark Knight'},
 {'imdb': {'votes': 1294646}, 'title': 'Inception'},
 {'imdb': {'votes': 1191784}, 'title': 'Fight Club'},
 {'imdb': {'votes': 1179033}, 'title': 'Pulp Fiction'},
 {'imdb': {'votes': 1109724},
  'title': 'The Lord of the Rings: The Fellowship of the Ring'},
 {'imdb': {'votes': 1087227}, 'title': 'Forrest Gump'},
 {'imdb': {'votes': 1081144},
  'title': 'The Lord of the Rings: The Return of the King'},
 {'imdb': {'votes': 1080566}, 'title': 'The Matrix'}]

<br><br><br>

### `$count`

This stage is to count the number of all output documents. Note that this operator can't be used to count groups of rows within the `$group` operator.

In [39]:
list(
    client["sample_mflix"]["movies"].aggregate(
        [
            {"$match": {"imdb.votes": {"$gt": 1000000}}},
            {"$count": "count_output_docs"},
        ]
    )
)

[{'count_output_docs': 12}]

<br><br><br>

### `$group`

This operator performs grouping of documents based on values in desired fields, and is very similar to the `GROUP BY` clause in SQL. Here is its syntax:

```
'$group':{
    '_id': field,
    aggregated_field: {accumulator: expression}
}
```

The `_id` field stores the distinct values found in `field`, based on which the grouping operation is performed. It acts as the primary key of the output documents.

In [40]:
list(
    client["sample_mflix"]["movies"].aggregate(
        [
            {'$group': {
                '_id': '$type',
                'count_each_type': {'$sum': 1}
            }}
        ]
    )
)

[{'_id': 'series', 'count_each_type': 254},
 {'_id': 'movie', 'count_each_type': 23276}]

<br><br><br>

### `$unwind`

Grouping by simple fields worked well. Things are a little bit trickier **if we want to aggregate based on values stored in arrays**.

For example, if we want to obtain the average IMDB score of movies played by each actor/actress, creating a grouping stage like the following wouldn't work:

```
{'$group': {
    '_id': '$cast',
    'avg_imdb_rating': {'$avg': '$imdb.rating'}
}}
```

The reason is, `$group` looks for exactly similar values stored in each field, and in this case it looks for similar documents that have exactly the cast, in the exact same order. This is obviously not what we want.

<br><br><br>

For these situations, there is an operator called `$unwind`, which basically **deconstructs** an array. The way it works is that **it creates copies of a document for each value of a desired array**. After doing this, the field containing an array gets converted to a simple field, and we can do the grouping as we are used to!

<br><br><br>

Let me show you what `$unwind` does for our very simple `students` collection of the `mds` database. Let's take a look at the collection again:

In [41]:
list(
    client['mds']['students'].find(
        filter={}
    )
)

[{'_id': ObjectId('6391cf15210a99d148d31810'),
  'name': 'Quan',
  'grades': [79, 87, 97]},
 {'_id': ObjectId('6391cf15210a99d148d31811'),
  'name': 'Varada',
  'grades': [75, 82, 90]},
 {'_id': ObjectId('6391cf15210a99d148d31812'),
  'name': 'Florencia',
  'grades': [92, 93, 77]}]

And now let's unwind the `grades` field:

In [42]:
list(
    client['mds']['students'].aggregate(
        [
            {'$unwind': '$grades'},
            {'$project': {'_id': 0}}
        ]
    )
)

[{'name': 'Quan', 'grades': 79},
 {'name': 'Quan', 'grades': 87},
 {'name': 'Quan', 'grades': 97},
 {'name': 'Varada', 'grades': 75},
 {'name': 'Varada', 'grades': 82},
 {'name': 'Varada', 'grades': 90},
 {'name': 'Florencia', 'grades': 92},
 {'name': 'Florencia', 'grades': 93},
 {'name': 'Florencia', 'grades': 77}]

Voila, we can now easily add a `$group` stage to, for example, compute the average of all grades for each student.

In [43]:
list(
    client['mds']['students'].aggregate(
        [
            {'$unwind': '$grades'},
            {'$group':{
                '_id': '$name',
                 'avg_grade': {'$avg': '$grades'}
            }}
        ]
    )
)

[{'_id': 'Varada', 'avg_grade': 82.33333333333333},
 {'_id': 'Quan', 'avg_grade': 87.66666666666667},
 {'_id': 'Florencia', 'avg_grade': 87.33333333333333}]